In [ ]:
!pip install biopython scikit-learn imbalanced-learn matplotlib seaborn --quiet
import pandas as pd
import numpy as np
from Bio import SeqIO
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from obliqueRF.models import ObliqueTreeClassifier
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from random import shuffle
def load_sequences(file_paths, window=150, stride=100):
    sequences, labels = [], []
    for label, path in file_paths.items():
        for record in SeqIO.parse(path, "fasta"):
            seq = str(record.seq).replace("*", "")
            for i in range(0, len(seq) - window + 1, stride):
                frag = seq[i:i+window]
                if len(frag) == window:
                    sequences.append(frag)
                    labels.append(label)
    combined = list(zip(sequences, labels))
    shuffle(combined)
    sequences, labels = zip(*combined)
    return list(sequences), list(labels)
# K-mer encoding
def get_kmers(seq, k=2):
    return [seq[i:i+k] for i in range(len(seq)-k+1)]
def build_kmer_features(sequences, k=2):
    all_kmers = set()
    kmer_counts = []
    for seq in sequences:
        kmers = get_kmers(seq, k)
        count = Counter(kmers)
        kmer_counts.append(count)
        all_kmers.update(count)
    all_kmers = sorted(list(all_kmers))
    df = pd.DataFrame(0, index=np.arange(len(sequences)), columns=all_kmers)
    for i, count in enumerate(kmer_counts):
        for kmer, freq in count.items():
            df.at[i, kmer] = freq
    return df
file_paths = {
    "Ebola": "/content/drive/MyDrive/sagar/Ebola_200.fasta",
    "Influenza_A": "/content/drive/MyDrive/sagar/Influenza_A_200.fasta",
    "Influenza_B": "/content/drive/MyDrive/sagar/Influenza_B_200.fasta",
    "SARS_CoV_2": "/content/drive/MyDrive/sagar/SARS_CoV_2_200.fasta",
    "Tuberculosis": "/content/drive/MyDrive/sagar/Tuberculosis_200.fasta",
    "Zika": "/content/drive/MyDrive/sagar/Zika_200.fasta",
    "Normal": "/content/drive/MyDrive/sagar/Human_200.fasta"
}
data, labels = load_sequences(file_paths, window=150, stride=100)
print(f"Augmented sequences: {len(data)}")
X = build_kmer_features(data, k=2)
y = pd.factorize(pd.Series(labels))[0]
selector = SelectKBest(mutual_info_classif, k=90)
X_selected = selector.fit_transform(X, y)
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X_selected, y)
X_train, X_test, y_train, y_test = train_test_split(
    X_balanced, y_balanced, test_size=0.2, stratify=y_balanced, random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
oblique_model = ObliqueTreeClassifier(
    max_depth=7,
    num_proj=1,
    criterion='gini',
    random_state=42
)
oblique_model.fit(X_train_scaled, y_train)
y_pred = oblique_model.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)
print(f"\n Oblique Tree Classifier Accuracy: {acc * 100:.2f}%")
print("\n Classification Report:\n")
print(classification_report(y_test, y_pred))

Augmented sequences: 5146

 Simulated Oblique Tree (Linear Node) Accuracy: 95.10%

 Classification Report:

              precision    recall  f1-score   support

           0       0.93      0.97      0.95       419
           1       0.88      0.86      0.87       419
           2       1.00      0.99      0.99       420
           3       0.89      0.91      0.90       420
           4       0.99      0.98      0.99       420
           5       0.96      0.95      0.96       420
           6       1.00      1.00      1.00       420

    accuracy                           0.95      2938
   macro avg       0.95      0.95      0.95      2938
weighted avg       0.95      0.95      0.95      2938

